In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/55.JPEG
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/130.JPEG
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/239.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/149.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/215.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/185.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/243.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/16.JPEG
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/153.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/232.png
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/189.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis/143.jpg
/kaggle/input/osteoporosis-knee-xray-dataset/osteop

In [2]:
import pandas as pd
import numpy as np
import os
import collections
import cv2
from sklearn.model_selection import train_test_split

In [3]:
os.listdir('/kaggle/input/osteoporosis-knee-xray-dataset/normal/')

['normal']

In [4]:
data = collections.defaultdict(list)
DIR = '/kaggle/input/osteoporosis-knee-xray-dataset/'
for i in os.listdir(DIR):
    dirs  = os.listdir(DIR + str(i)+'/'+str(i))
    print(f"Processing : {i}")
    for img in dirs:
        temp_dir = DIR + str(i)+'/'+str(i)+'/'+img
        data[i].append(cv2.resize(cv2.imread(temp_dir), (100,100)))
        

Processing : osteoporosis
Processing : normal


In [5]:
len(os.listdir('/kaggle/input/osteoporosis-knee-xray-dataset/osteoporosis/osteoporosis'))

186

In [6]:
X = []
Y = []
count = 0
for i in data.keys():
    print("Images in {} ==> {}".format(i,len(data[i])))
    X += data[i]
    for _ in range(len(data[i])):
        Y.append(count)
    count += 1

Images in osteoporosis ==> 186
Images in normal ==> 186


In [7]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [8]:
X = np.array(X)
Y = np.array(Y)

In [9]:
Y = to_categorical(Y)

In [10]:
xtrain,xtest, ytrain, ytest = train_test_split(X,Y,test_size=.10, random_state = 0) 

In [11]:
model=Sequential()
model.add(Conv2D(64,(3,3),activation='relu',input_shape=(100,100,3)))
#pooling layer
model.add(MaxPooling2D(3,3))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu'))
#pooling layer
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        3

2022-10-04 14:27:32.812742: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [12]:
model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(xtrain,ytrain,epochs=30,batch_size=16,validation_data = (xtest,ytest))

2022-10-04 14:27:33.130712: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
21/21 [==============================] - 3s 111ms/step - loss: 0.7955 - accuracy: 0.6108 - val_loss: 3.6241 - val_accuracy: 0.4474
Epoch 2/30
21/21 [==============================] - 2s 98ms/step - loss: 0.5763 - accuracy: 0.7365 - val_loss: 2.8670 - val_accuracy: 0.5263
Epoch 3/30
21/21 [==============================] - 2s 101ms/step - loss: 0.5224 - accuracy: 0.7395 - val_loss: 0.9987 - val_accuracy: 0.6316
Epoch 4/30
21/21 [==============================] - 2s 101ms/step - loss: 0.4765 - accuracy: 0.7635 - val_loss: 1.0632 - val_accuracy: 0.5263
Epoch 5/30
21/21 [==============================] - 2s 101ms/step - loss: 0.4565 - accuracy: 0.7904 - val_loss: 0.8677 - val_accuracy: 0.6316
Epoch 6/30
21/21 [==============================] - 2s 98ms/step - loss: 0.4586 - accuracy: 0.7665 - val_loss: 0.6239 - val_accuracy: 0.7632
Epoch 7/30
21/21 [==============================] - 2s 100ms/step - loss: 0.4141 - accuracy: 0.8144 - val_loss: 0.6394 - val_accuracy: 0.8158
Epoch 8/

In [14]:
model.fit(xtrain,ytrain,epochs=30,batch_size=16,validation_data = (xtest,ytest))

Epoch 1/30
21/21 [==============================] - 2s 103ms/step - loss: 0.1696 - accuracy: 0.9371 - val_loss: 1.5431 - val_accuracy: 0.7105
Epoch 2/30
21/21 [==============================] - 2s 101ms/step - loss: 0.1722 - accuracy: 0.9311 - val_loss: 1.4692 - val_accuracy: 0.6842
Epoch 3/30
21/21 [==============================] - 2s 97ms/step - loss: 0.1302 - accuracy: 0.9251 - val_loss: 1.4793 - val_accuracy: 0.6053
Epoch 4/30
21/21 [==============================] - 2s 98ms/step - loss: 0.1135 - accuracy: 0.9521 - val_loss: 1.4476 - val_accuracy: 0.7105
Epoch 5/30
21/21 [==============================] - 2s 102ms/step - loss: 0.1364 - accuracy: 0.9491 - val_loss: 1.6211 - val_accuracy: 0.6579
Epoch 6/30
21/21 [==============================] - 2s 102ms/step - loss: 0.1201 - accuracy: 0.9491 - val_loss: 1.8150 - val_accuracy: 0.6053
Epoch 7/30
21/21 [==============================] - 2s 100ms/step - loss: 0.1587 - accuracy: 0.9251 - val_loss: 1.8433 - val_accuracy: 0.6842
Epoch 8/

In [15]:
model.fit(xtrain,ytrain,epochs=10,batch_size=16,validation_data = (xtest,ytest))

Epoch 1/10
21/21 [==============================] - 2s 100ms/step - loss: 0.0734 - accuracy: 0.9731 - val_loss: 1.8397 - val_accuracy: 0.6579
Epoch 2/10
21/21 [==============================] - 2s 98ms/step - loss: 0.0630 - accuracy: 0.9671 - val_loss: 2.0392 - val_accuracy: 0.7105
Epoch 3/10
21/21 [==============================] - 2s 104ms/step - loss: 0.0563 - accuracy: 0.9731 - val_loss: 2.5451 - val_accuracy: 0.7105
Epoch 4/10
21/21 [==============================] - 2s 101ms/step - loss: 0.0502 - accuracy: 0.9731 - val_loss: 2.4494 - val_accuracy: 0.7368
Epoch 5/10
21/21 [==============================] - 2s 104ms/step - loss: 0.0839 - accuracy: 0.9641 - val_loss: 2.5793 - val_accuracy: 0.7105
Epoch 6/10
21/21 [==============================] - 2s 100ms/step - loss: 0.0747 - accuracy: 0.9581 - val_loss: 2.3088 - val_accuracy: 0.7105
Epoch 7/10
21/21 [==============================] - 2s 96ms/step - loss: 0.0469 - accuracy: 0.9671 - val_loss: 2.3773 - val_accuracy: 0.7105
Epoch 8/

In [16]:
model.save("/kaggle/working/Osteoporosis_73.h5")

In [17]:
from sklearn.metrics import classification_report

In [18]:
print(classification_report(np.argmax(ytest,axis=1),np.argmax(model.predict(xtest),axis=1)))

              precision    recall  f1-score   support

           0       0.70      0.76      0.73        21
           1       0.67      0.59      0.62        17

    accuracy                           0.68        38
   macro avg       0.68      0.68      0.68        38
weighted avg       0.68      0.68      0.68        38

